In [21]:
import re

def is_valid(text: str) -> bool:
    """
    判断该微博是否为有效微博，依据为广告、转载、营销相关的关键词
    如果微博中出现了关键词，则认为是无效微博，直接删除

    Args:
        text (str): 一条微博的文本内容

    Returns:
        bool: 微博是否是否有效
    """

    if len(text.strip()) <= 1:
        return False
    
    # 广告、转载相关关键词
    ad_keywords = [
        '点开红包', '现金红包', '好礼', '网页链接', 
        '我在参与', '连续签到', '粉打卡', '年度歌曲', 
        '免费围观', '关注超话', "蚂蚁庄园：", "森林驿站", 
        "头条文章", "注册微博", "注册微博", "闲鱼发布",
        "闲鱼号", "头像挂件"
    ]

    # 微博中含有任意一个关键词，则认为无效
    if any(kw in text for kw in ad_keywords):
        # print(f"微博“{text}”为抽奖、广告、转载等微博，删除")
        return False
    
    return True



def clean_text(text):
    # 去除分享来源信息（直接将“分享自”及其后面的所有信息去除）
    text = re.sub(r'(?:[（(])?分享自(?!己).*$', '', text)

    # 去除“分享图片/视频”文本
    text = re.sub(r'分享(图片|视频)', '', text)
    
    # 去除多余空格
    text = re.sub(r'[\s]+', ' ', text)
    
    # 去除纯表情符号的内容
    if len(re.sub(r'[\s]', '', text)) <= 1:
        return ''
    
    return text.strip()


import json

file_name = "test100.json"

with open(file_name, 'r', encoding='utf-8') as f:
    user_dict:dict = json.load(f)

In [22]:
users_to_remove = []

for user_id, user_info in user_dict.items():
    weibos = user_info["weibo"]
    for weibo in weibos:
        weibo["text"] = clean_text(weibo["text"])
    old = len(weibos)

    # 删除文本长度过短和抽奖打卡微博
    weibos = [weibo for weibo in weibos if is_valid(weibo["text"])]
    new = len(weibos)
    user_info["weibo"] = weibos
    rmv = old - new
    if new < 10:
        print(f"删除了用户{user_id}的{rmv}条无效微博，剩余微博数（{new}）过少，需要将该用户清洗出数据集")
        users_to_remove.append(user_id)
            
for user_id in users_to_remove:
    del user_dict[user_id]

print(f"删除了{len(users_to_remove)}个用户")

删除了用户1001148342的12条无效微博，剩余微博数（0）过少，需要将该用户清洗出数据集
删除了用户1011491753的21条无效微博，剩余微博数（5）过少，需要将该用户清洗出数据集
删除了用户1058548740的50条无效微博，剩余微博数（0）过少，需要将该用户清洗出数据集
删除了用户1144313674的48条无效微博，剩余微博数（2）过少，需要将该用户清洗出数据集
删除了4个用户


In [23]:
save_file = "test100cleaned.json"

with open(save_file, 'w', encoding='utf-8') as f:
    json.dump(user_dict, f, ensure_ascii=False, indent=4)

In [53]:

def truncate_text(text: str, max_len: int):
    if len(text) <= max_len:
        return text
    return text[:max_len] + "..."

def sample_weibos(user_dict, max_weibo_len: int = 200, max_weibo_count: int = 20):
    sample_weibo_dict = {}
    for user_id, user_info in user_dict.items():
        weibo_list = user_info["weibo"]
        if len(weibo_list) <= max_weibo_count:
            step = 1
        else:
            step = len(weibo_list) // (max_weibo_count - 1)
        
        sample_weibo_list = [
            truncate_text(weibo["text"], max_weibo_len)
            for weibo in weibo_list[::step][:max_weibo_count]
        ]

        sample_weibo_dict[user_id] = sample_weibo_list
    return sample_weibo_dict

sample_weibo_dict = sample_weibos(user_dict)


In [54]:
from openai import OpenAI

with open("../weibo_config.json", 'r', encoding='utf-8') as f:
    config = json.load(f)

llm_config = config["LLM_API"]["DeepSeek-v3"]


prompt = """
你是一名专业的社交媒体信息分析师，请根据用户发布的微博内容判断是否为生活分享账号，需满足以下至少2项特征：
1. 包含具体生活场景/人物互动/个人经历
2. 使用第一人称主观表达（如"我"的感受/经历）
3. 内容呈现非结构化自然叙述（非列表/教程/资讯格式）
4. 涉及日常活动（饮食/出行/家庭/宠物等）

需排除以下特征账号：
• 垂直领域专业内容（医疗/法律/金融等）
• 商品交易/广告推广信息
• 抽象理论/鸡汤语录/政策转载
接下来我将给出一个JSON文件，其中键是用户ID，值是经过采样后的微博内容。
请直接为我返回数字“0”或“1”，“1”代表用户是生活分享账号，“0”代表用户不是生活分享账号。除此之外无需返回其他内容。
"""

def validity_analysis(llm_config: dict, prompt: str, user_weibo):
    base_url = llm_config["base_url"]
    model = llm_config["model"]
    api_key = llm_config["api_key"]
    if isinstance(api_key, list) and len(api_key) == 1:
        api_key = api_key[0]
    if not isinstance(user_weibo, str):
        if isinstance(user_weibo, dict):
            user_weibo = json.dumps(user_weibo, ensure_ascii=False)
        else:
            user_weibo = str(user_weibo)
        
    client = OpenAI(api_key=api_key, base_url=base_url)


    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_weibo},
        ],
        temperature=1.0, 
        stream=False
    )

    return response.choices[0].message.content

In [ ]:
def multi_model_validity_analysis(user_weibo_dict):
    # 存储所有用户的分析结果
    all_results = {}
    
    # 定义要使用的三个模型配置
    models = [
        config["LLM_API"]["DeepSeek-v3"],
        config["LLM_API"]["Qwen-Plus"],
        config["LLM_API"]["Moonshot-v1"]
    ]
    
    # 对每个用户进行分析
    for user_id, weibo_list in user_weibo_dict.items():
        user_results = []
        # 构造单个用户的测试数据
        test_data = {user_id: weibo_list}
        
        # 使用三个不同的模型进行分析
        for model_config in models:
            try:
                result = validity_analysis(model_config, prompt, test_data)
                # 确保结果是 "0" 或 "1"
                if result.strip() in ['0', '1']:
                    user_results.append(int(result.strip()))
                else:
                    model = model_config["model"]
                    print(f"模型 {model} 分析用户 {user_id} 时的返回结果：{result} 存在问题")
                    user_results.append(None)  # 标记无效结果
            except Exception as e:
                print(f"分析用户 {user_id} 时发生错误: {str(e)}")
                user_results.append(None)
        
        # 存储该用户的所有模型结果
        all_results[user_id] = user_results
        
        # 打印进度
        print(f"已完成用户 {user_id} 的分析，结果: {user_results}")
    
    return all_results


# 使用示例
results = multi_model_validity_analysis(sample_weibo_dict)

已完成用户 1000129923 的分析，结果: [1, 1, 1]
已完成用户 1001430012 的分析，结果: [1, 1, 1]
已完成用户 1005071045 的分析，结果: [0, 1, 0]
已完成用户 1005603481 的分析，结果: [1, 1, 1]
已完成用户 1006429701 的分析，结果: [1, 1, 1]
已完成用户 1007111115 的分析，结果: [0, 0, 0]
已完成用户 1010165301 的分析，结果: [1, 1, 1]
已完成用户 1011074602 的分析，结果: [1, 1, 1]
已完成用户 1011874467 的分析，结果: [0, 0, 0]
已完成用户 1013933125 的分析，结果: [1, 1, 1]
已完成用户 1015090037 的分析，结果: [1, 1, 1]
已完成用户 1015302932 的分析，结果: [0, 0, 0]
已完成用户 1015581375 的分析，结果: [1, 1, 1]
已完成用户 1022600720 的分析，结果: [0, 0, 0]
已完成用户 1024611264 的分析，结果: [0, 1, 1]
已完成用户 1025714307 的分析，结果: [1, 1, 1]
已完成用户 1032458983 的分析，结果: [0, 1, 0]
已完成用户 1033961260 的分析，结果: [1, 1, 1]
已完成用户 1035490573 的分析，结果: [1, 1, 1]
已完成用户 1039197400 的分析，结果: [1, 1, 1]
已完成用户 1040345572 的分析，结果: [1, 1, 1]
已完成用户 1041044002 的分析，结果: [1, 1, 1]
已完成用户 1043086511 的分析，结果: [1, 1, 1]
已完成用户 1044396822 的分析，结果: [1, 1, 1]
已完成用户 1044688085 的分析，结果: [0, 1, 0]
已完成用户 1046200023 的分析，结果: [1, 1, 1]
已完成用户 1050102111 的分析，结果: [1, 1, 1]
已完成用户 1052264532 的分析，结果: [1, 1, 1]
已完成用户 1055933083 的分析

In [89]:

users_to_remove = []
error_users = []
users_to_examine = []
counter = {0:0, 1:0, 2:0, 3:0}
threshold = 2
# 打印结果统计
for user_id, result in results.items():
    if None in result:
        error_users.append(user_id)
        continue
    
    score = sum(result)
    counter[score] += 1
    if score < threshold:
        users_to_remove.append(user_id)
    elif threshold <= score < 3:
        users_to_examine.append(user_id)
    # print(f"用户 {user_id}: 模型结果 = {result}, 得分 = {score}")
print(counter)

{0: 19, 1: 7, 2: 6, 3: 64}


In [87]:
for user_id in users_to_remove:
    print(f"{user_id}:")
    print(f"result：{results[user_id]}")
    # print(sample_weibo_dict[user_id])
    for weibo in sample_weibo_dict[user_id]:
        print(weibo)
    print('-'*50)

1005071045:
result：[0, 1, 0]
#王鹤棣哈尔滨啤酒代言人# 嘿！我在#星品影响力#上邂逅了一个疯狂心动的星品——@哈尔滨啤酒 @王鹤棣_Dylan，快来看看，说不定你的心动之选也在这里哦！ 星品影响力新宣星品
#生活手记#晚安！
我在#星品影响力#上为 @哈尔滨啤酒 X @王鹤棣_Dylan 连续送心7天，获得“星品热爱官”称号！快来一起pick你心动的星品吧～ 星品影响力
#生活手记#晚安！
#生活手记#晚安！
#王鹤棣喊你入巧门#开屏看到粉粉的王鹤棣啦
#生活手记#晚安！
#生活手记#晚安！
#生活手记#晚安！
#生活手记#午后阳光正好，微风不燥
悲桑，殉了6只小🐭，此鰐果然很惡
#生活手记#晚安！
#生活手记#晚安！
#生活手记#晚安！
#你好星期六# 我的评分： 不用心好好做节目，有dcx在都不会看了
#生活手记#晚安！
--------------------------------------------------
1007111115:
result：[0, 0, 0]
普通人真的不该炒股吗？其实未必！ 常有人告诫普通人别炒股，然而，经历过创业且有所成就的人，往往会鼓励大家尽早涉足炒股领域，确切地说，是尽早开启投资之路。这一观点与我们所受的传统教育存在偏差。传统教育倡导一分能力做一分事，但现实中，人们常常是凭借一分能力去挑战三分事，即便付出财产与时间的代价，却也收获了额外的能力与见识。毕竟，亲身经历的教训，远比他人的教导更刻骨铭心。 所以，对于那些没有家业可继承，...
不能太急，翻身也许就在两、三年时间里。 好好沉淀。
真正厉害的人像开了省电模式，少掺和八卦，少抱怨命运，能量都用来滋养自己。这种状态下眼神会变锐利，心思会变通透，小人都拿他没办法，烂事来了不攻自破。 反观能量低的人：疯狂刷存在感、四处求认同，闲聊是非八卦。等能量耗光时，看人看事全是雾里看花，连基本判断力都会失灵。 记住：你每向内收一分能量，气场就厚一寸；每向外求一次认可，元神就弱三分。
人类想要不被淘汰只有一条路：一辈子不断学习，不断改变，不断打造全新的自己。这种改变不只是知识，还包括自我认知和性格。 ——尤瓦尔·赫拉利
钱，不是赚来的，越想赚钱的人，赚不到钱。 钱是帮他人解决问题后给你的回报，什么时候明白了这句话，你就开始赚钱了。
德国哲学家尼采说： “我

In [92]:
for user_id in users_to_examine:
    print(f"{user_id}:")
    print(f"result：{results[user_id]}")
    # print(sample_weibo_dict[user_id])
    for index, weibo in enumerate(sample_weibo_dict[user_id]):
        print(f"{index+1}. {weibo}")
    print('-'*50)

1024611264:
result：[0, 1, 1]
1. 最近没有什么工作动力
2. #阿暖看文记录# 暗恋偏差by山河南渡 3.5星 前面拉拉扯扯来来回回有点烦，攻太幼稚，但整体又不太调动情绪，没文看的时候可以随便看看
3. #阿暖看文记录# 被冒充的丈夫by升兮兮 3.5-4星 微恐小短篇，因为短所以剧情还蛮紧凑的 前面感觉每个人都好癫，结尾救回来一点吧
4. 我好想戴新甲片出去玩，但我的休息时间谁也约不到
5. #阿暖看文记录# 有效真香by酌青栀 po文 剧情3.5🥩3.5，但是有剧情有🥩所以4星 开始是女主主动，偏女性向，整体还行，吃肉可看
6. 能不能下定金第二天就补尾款第三天就发货啊
7. 听我妈讲长辈亲戚的八卦真的很炸裂
8. 想说去感受下万圣节活动的环球影城，看了下鬼屋第一视角，根本不敢去
9. #阿暖看文记录# 一篇替身文by岫青晓白 3-3.5星 有点套路没有新意，属于是看开头就能猜到结尾的，打发时间看
10. 摇篮曲直接单jsk快1500，这样一比花🦐899的jsk问题也没那么大了
11. 短剧能不能别再找发面馒头演男主 瘦的化妆还能救一救，发面馒头那就是无解纯丑啊
12. 山花烂漫时也太好哭了
13. 周末玩景点就是给自己找罪受，玩不回票价的十分之一🙅 开封·万岁山大宋武侠城
14. 可能是年龄到了，小巷人家的后劲比永夜星河足，好羡慕主角生活 （婆媳和重男轻女除外）
15. #阿暖看文记录# 南方海啸by卡比丘 3.5星 有点短，细水长流型的，不太轰轰烈烈也没有狗血，想看官宣也木有 喜欢番外
16. #周芯竹回应分手事件#我真是无语😑
17. #阿暖看文记录# 囚于永夜by麦香鸡呢 4-4.5⭐ 前面4.5，后面因为没连载完断断续续看接不上了没有开始一口气看的快乐了 好看，不过更喜欢欲言难止
18. 周大福新款的蛇也太好看了，恨自己不属蛇，不然咬咬牙买了
19. #阿暖追剧记录#【看完】 纪录片 《闪闪的儿科医生2》一如既往好看好哭 《守护解放西5》比4好看，4断断续续好像没看完 电视 《永夜星河》妆造和演技好，结局不喜欢🙅 电影 《好东西》好看很喜欢 【在看】 电视 《九重紫》好看爱看 《猎罪图鉴2》好看，但是这季目前推理略简单，好容易猜出来凶手 纪录片 《梦想改造家11》居然这么多季了，做手工时候当背景音挺好 《是坏情绪啊，没关系

In [2]:
dct = {'a': 1, 'b': 2}

list(dct.items())

[('a', 1), ('b', 2)]